# Bibliotecas

In [ ]:
import yfinance as yf
from matplotlib import pyplot as plt
import pandas as pd
import mplfinance as mplf
from os.path import exists
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyRegressor

# Obter Dados

In [ ]:
if not exists('historico.csv'):
    hist = yf.download('ITSA4.SA', period='10y', interval='1d')
    hist.to_csv('historico.csv')
else:
    hist = pd.read_csv('historico.csv', index_col='Date', parse_dates=True)  
hist

# Tratamento de Dados

### Ajuste de Preços

In [ ]:
histAdj = hist.copy()
histAdj = histAdj.drop(columns=['Adj Close', 'Volume'])
adj = hist['Adj Close'] / hist['Close']
histAdj = histAdj.multiply(adj, axis=0)
histAdj

### Gerando colunas com preços passados

In [ ]:
for i in range(100):
    histAdj['Close'+str(i+1)] = histAdj['Close'].shift(i+1)
    histAdj['Open'+str(i+1)] = histAdj['Open'].shift(i+1)
    histAdj['High'+str(i+1)] = histAdj['High'].shift(i+1)
    histAdj['Low'+str(i+1)] = histAdj['Low'].shift(i+1)
histAdj = histAdj.dropna()
histAdj

# Teste de Modelo de Regressão

### Separar dados de treino e teste

In [ ]:
x = histAdj.drop(columns=['Open', 'Close', 'High', 'Low'])
y = histAdj[['Open', 'High', 'Low', 'Close']]
splitDays = -250
x_train, x_test, y_train, y_test = x.iloc[:splitDays], x.iloc[splitDays:], y.iloc[:splitDays], y.iloc[splitDays:]
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

### Lazy Regressor

In [ ]:
from lazypredict.Supervised import LazyRegressor

# LazyRegressor().fit(x_train, x_test, y_train, y_test)

### Criando um dos modelos encontrados

In [ ]:
from sklearn.linear_model import OrthogonalMatchingPursuit
from math import sqrt

modelo = OrthogonalMatchingPursuit()
modelo.fit(x_train, y_train)
y_train_predict = modelo.predict(x_train)
y_test_predict = modelo.predict(x_test)
print('Treino R2:', r2_score(y_train, y_train_predict))
print('Teste R2:', r2_score(y_test, y_test_predict))
print('Treino RMSE:', sqrt(mean_squared_error(y_train, modelo.predict(x_train))))
print('Teste RMSE:', sqrt(mean_squared_error(y_test, modelo.predict(x_test))))


# Visualização

### Dados do Histórico

In [ ]:
fig = mplf.figure(figsize=(15, 4))
mplf.plot(y_test, type='candle', ax=fig.add_subplot(2, 1, 1))
prediction = modelo.predict(x_test)
dfPrediction = pd.DataFrame(prediction, y_test.index, y_test.columns)
mplf.plot(dfPrediction, type='candle', ax=fig.add_subplot(2, 1, 2))
dif = y - dfPrediction
plt.figure(figsize=(15, 4))
plt.plot(dif['Close'])
plt.show()

# Prever ontem, hoje e amanhã

In [77]:
histAtual = yf.download('ITSA4.SA', period='103d', interval='1d')
adj = histAtual['Adj Close'] / histAtual['Close']
histAtual = histAtual.drop(columns=['Adj Close', 'Volume'])
histAtual = histAtual.multiply(adj, axis=0)
amanhaTimeStamp = histAtual.iloc[[-1]].index + pd.Timedelta(days=1)
amanhaDados = {
    'Open': 0,
    'High': 0,
    'Low': 0,
    'Close': 0
}
histAtual = pd.concat([histAtual, pd.DataFrame([amanhaDados], index=amanhaTimeStamp)])
for i in range(100):
    histAtual['Close'+str(i+1)] = histAtual['Close'].shift(i+1)
    histAtual['Open'+str(i+1)] = histAtual['Open'].shift(i+1)
    histAtual['High'+str(i+1)] = histAtual['High'].shift(i+1)
    histAtual['Low'+str(i+1)] = histAtual['Low'].shift(i+1)
histAtual = histAtual.dropna()
x = histAtual.drop(columns=['Open', 'High', 'Low', 'Close'])
previsao = pd.DataFrame(modelo.predict(x), x.index, ['Open', 'High', 'Low', 'Close'])
print('Dados:\n', y.iloc[-2:])
print('Previsão:\n', previsao)

[*********************100%***********************]  1 of 1 completed
Dados:
             Open  High  Low  Close
Date                              
2023-06-26  9.65  9.72 9.57   9.66
2023-06-28  9.50  9.57 9.47   9.54
Previsão:
             Open  High  Low  Close
Date                              
2023-06-26  9.66  9.72 9.54   9.64
2023-06-28  9.65  9.71 9.57   9.64
2023-06-29  9.57  9.65 9.49   9.58
